**REMEMBER**: add the config file `~/.pylidcrc` in the root directory that specifies where the dataset is store on your machine
```
[dicom]
path = .../path/LIDC-IDRI
warn = True
```

In [2]:
import pylidc as pl
import numpy as np
import matplotlib.pyplot as plt

# to avoid error in pylidc due to deprecated types:
np.int = int
np.float = float

# `Scan` class
[see class docs here](https://pylidc.github.io/scan.html)
- The `pylidc.Scan` class holds some (but not all!) of the DICOM attributes associated with the CT scans in the LIDC dataset. These attributes can be used to query the data.
- A `pylidc.Scan` object has zero or more pylidc.Annotation objects, which are radiologist annotations of lung nodules found in the scan:

In [ ]:
# get the first scan
scan = pl.query(pl.Scan).first()
print(scan.study_instance_uid)
print(scan.series_instance_uid)
print(scan.patient_id)
print(scan.slice_thickness)
print(scan.zvals)

In [ ]:
# how to filter/query scans
scans = pl.query(pl.Scan).filter(pl.Scan.slice_thickness <= 1,
                                 pl.Scan.pixel_spacing <= 0.6).all()
len(scans)

In [ ]:
# Descriptics
total_scan_counts = pl.query(pl.Scan).count()
scans_with_annotations = pl.query(pl.Scan).filter(pl.Scan.annotations.any()).count()
print("Total number of scans: ", total_scan_counts)
print("Number of scans with annotations: ", scans_with_annotations)

In [ ]:
# to open the plot interactively:
%matplotlib qt

# utils
scan = pl.query(pl.Scan).first() # get a scan

# images = scan.load_all_dicom_images(verbose=False) # load all DICOM images for this scan
# print(len(images))

# scan_dir = scan.get_path_to_dicom_files() # get the path to the DICOM files for this scan

# return the scan as a 3D numpy array
# volumne = scan.to_volume(verbose=False)

# cluster the annotations
nodules = scan.cluster_annotations()

# visualise the scan

scan.visualize(annotation_groups=nodules)

%matplotlib inline

# `Annotation` class
[see class docs here](https://pylidc.github.io/annotation.html)

The Nodule model class holds the information from a single physicians annotation of a nodule >= 3mm class with a particular scan. A nodule has many contours, each of which refers to the contour drawn for nodule in each scan slice.

Refer to the documentation link above for semantic describtions of what the different values mean.

In [ ]:
# Annotation features
pl.annotation_feature_names
# NOTE: see semantic meaning of the feature values in the pylidc documentation

In [ ]:
ann = pl.query(pl.Annotation)\
        .filter(pl.Annotation.spiculation > 3).first()

# print the feature values
ann.print_formatted_feature_table()

In [ ]:
# Descriptics
annotations_count = pl.query(pl.Annotation).count()
print("Total number of annotations: ", annotations_count)

print("----------------------------------------")

malignancy_counts = []
print("Number of annotations by malignancy:")
for i in range(5):
    ann = pl.query(pl.Annotation).filter(pl.Annotation.malignancy == i)
    malignancy_counts.append(ann.count())
    # print(f"Malignancy {i +1}: {ann.count()}")

r = range(1, 5 + 1)
plt.bar(r, malignancy_counts)
for i, count in enumerate(malignancy_counts):
    plt.text(i + 1, count, str(count), ha='center', va='bottom')
plt.xlabel("Malignancy of a Nodule\n(1: Highly unlikely,..., 5: Highly Suspicious)")
plt.show()


print("----------------------------------------")

print("Number of annotations by subtlety:")
for i in range(5):
    ann = pl.query(pl.Annotation).filter(pl.Annotation.subtlety == i)
    print(f"Subtlety {i +1}: {ann.count()}")

In [ ]:
malignant_count = pl.query(pl.Annotation).filter(pl.Annotation.malignancy <= 4).count()
ambiguous_count = pl.query(pl.Annotation).filter(pl.Annotation.malignancy == 3).count()
benign_count = pl.query(pl.Annotation).filter(pl.Annotation.malignancy <= 2).count()

plt.title("Annotations by Malignancy\n(in accordance with VAE papers binning)")
plt.bar(["Malignant", "Ambiguous", "Benign"], [malignant_count, ambiguous_count, benign_count])
for i, count in enumerate([malignant_count, ambiguous_count, benign_count]):
    plt.text(i, count, str(count), ha='center', va='bottom')
plt.show()

# Utils - Annotation consensus and Volume Viewer
The `pylidc.utils.consensus()` utility accepts a list of annotations to produce a single boolean-valued volume of those annotations. It also returns the individual boolean masks for each Annotation, placed in a common frame of reference – i.e., a common bounding box, which is also returned.

In [ ]:
# see https://pylidc.github.io/utils.html#pylidc.utils.consensus for utils documentation
from pylidc.utils import consensus
from skimage.measure import find_contours

def show_segmentation_consensus(scan: pl.Scan, nodule_idx: int, clevel: float = 0.5) -> None:
    """
    Display the consensus segmentation for a nodule in a scan.

    Args:
        @scan: The pylidc.Scan object.
        @nodule_idx: The index of the nodule in the scan.
        @clevel: The consensus fraction. For example, if clevel=0.5, then a voxel will have value 1 (True) in the returned boolean volume when at least 50% of the annotations have a value of 1 at that voxel, and 0 (False) otherwise.
        @pad: The number of slices to pad the volume by in the z-direction.
    """
    # Query for a scan, and convert it to an array volume.
    vol = scan.to_volume(verbose=False)

    # Cluster the annotations for the scan, and grab one.
    nods = scan.cluster_annotations()
    anns = nods[nodule_idx] # TODO maybe this should be the nodule id ??

    # Perform a consensus consolidation and 50% agreement level.
    # We pad the slices to add context for viewing.
    cmask, cbbox, masks = consensus(anns, clevel=clevel, pad=[(20,20), (20,20), (0,0)])

    # Get the central slice of the computed bounding box.
    k = int(0.5*(cbbox[2].stop - cbbox[2].start))

    # Set up the plot.
    fig,ax = plt.subplots(1,1,figsize=(5,5))
    ax.imshow(vol[cbbox][:,:,k], cmap=plt.cm.gray, alpha=0.5)

    # Plot the annotation contours for the kth slice.
    colors = ['r', 'g', 'b', 'y']
    for j in range(len(masks)):
        for c in find_contours(masks[j][:,:,k].astype(float), 0.5):
            label = "Annotation %d" % (j+1)
            plt.plot(c[:,1], c[:,0], colors[j], label=label)

    # Plot the 50% consensus contour for the kth slice.
    for c in find_contours(cmask[:,:,k].astype(float), 0.5):
        plt.plot(c[:,1], c[:,0], '--k', label='50% Consensus')

    ax.axis('off')
    ax.legend()
    plt.tight_layout()
    plt.show()

show_segmentation_consensus(scan, nodule_idx=1)

In [ ]:
from pylidc.utils import volume_viewer

# to open the plot interactively:
%matplotlib qt

ann = pl.query(pl.Annotation).first()
vol = ann.scan.to_volume(verbose=False)
np.float = float
padding = 120.0

mask = ann.boolean_mask(pad=padding)
bbox = ann.bbox(pad=padding)

volume_viewer(vol[bbox], mask, ls='-', lw=2, c='r')

# to close the plot again:
%matplotlib inline

# Util funcs

In [ ]:
# all util functions
def get_scans_by_patient_id(patient_id: str) -> list[pl.Scan]:
    """Returns all scans for a given patient_id"""
    scans = pl.query(pl.Scan).filter(pl.Scan.patient_id == patient_id).all()
    return scans


def get_annotations_by_scan_id(scan_id: int) -> list[pl.Annotation]:
    """Returns all annotations for a given scan_id"""
    annotations = pl.query(pl.Annotation).filter(pl.Annotation.scan_id == scan_id).all()
    return annotations


def get_annotations_by_scan(scan: pl.Scan) -> list[pl.Annotation]:
    """Returns all annotations for a given scan object"""
    return scan.annotations

def get_nodules_by_scan(scan: pl.Scan) -> list[pl.Annotation]:
    """Returns all nodules for a given scan object"""
    return scan.cluster_annotations()


pid_scans = get_scans_by_patient_id("LIDC-IDRI-0010")
scan_annotations = get_annotations_by_scan_id(pid_scans[0].id)
scan_nodules = get_nodules_by_scan(pid_scans[0])

# get the malignancy of each annotation:
[s.malignancy for s in scan_annotations]